# AWS Documentation RAG-QA

## Notebook Purpose
Data Anaysis for Chunking and Indexing
 
## Tasks
- Data Chunk
- Indexing

## Notable TODOs:
- Evaluation

---

# Setup

In [1]:
from pathlib import Path
from IPython import get_ipython

ipython = get_ipython()
if Path.cwd().name == "notebooks":
	%cd ..

## Extensions
if 'autoreload' not in ipython.extension_manager.loaded:
	%load_ext autoreload
else:
	%reload_ext autoreload

if 'dotenv' not in ipython.extension_manager.loaded:
	%load_ext dotenv
else:
	%reload_ext dotenv

# if 'cudf.pandas' not in get_ipython().extension_manager.loaded:
# 	%load_ext cudf.pandas

%autoreload 2
%dotenv

/home/leobit/Development/Projects/aws-doc-ragqa/research


In [2]:
## Config / Utils
import config as cfg
from IPython.core.magic import register_cell_magic
from utils import save_obj, load_obj, run_api

@register_cell_magic
def pybash(line, cell):
	'''Runs a magic bash with Python Variables'''
	ipython.run_cell_magic('bash', '--no-raise-error', cell.format(**globals()))

### System Infomation

In [2]:
%%pybash
bash {cfg.path.scripts}/notebook_info.sh

## GLOBAL INFO
Conda Python Version: 

3.12.9.final.0
Conda Base Path: /opt/miniconda3
Conda Base Version: 25.1.1

## ENVIRONMENT INFO
Active Environment: aws-doc-ragqa
Environment Python Version: Python 3.11.13
Environment Python Path: /opt/miniconda3/envs/aws-doc-ragqa/bin/python
Environment IPython Version: 9.1.0
Environment IPykernel Version: 6.29.5

## GPU INFO:
CUDA Device Initialized 

/home/leobit/Development/Projects/aws-doc-ragqa/research/scripts/notebook_info.sh: line 21: numba: command not found




GPU Info: Failed to initialize NVML: N/A
Failed to properly shut down NVML: N/A



---

# Data Chunk

- Strategy: Semantic and Hierarchical Chunking  
- Parser: MarkdownNodeParser

#### Testing some chunking documents

In [3]:
import pandas as pd
from IPython.display import Markdown, display
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import MarkdownNodeParser

In [4]:
category_files_df = load_obj(cfg.path.data.interim / "category_files_df_v1.pickle", as_df=True)

In [5]:
category_files_df.category.unique()

array(['properties', 'resource', 'how-to-guide', 'tutorial', 'concepts',
       'security', 'geospatial'], dtype=object)

In [6]:
category_files_df.iloc[0]["path"]

PosixPath('data/raw/aws_doc_batch_1/aws-properties-events-rule-sagemakerpipelineparameter.md')

In [7]:
df = category_files_df.copy()

def get_category(df: pd.DataFrame, path: Path):
	category = df[df.path == path]["category"].values[0]
	return category

def get_metadata(file_path):
	file_path = Path(file_path)
	return {"file_name": file_path.name, "category": get_category(df, file_path)}

def check_parser(df: pd.DataFrame, cat: str, qtd_docs: int = 1):
	paths = df[df.category==cat].sample(qtd_docs)["path"].tolist()
	docs = SimpleDirectoryReader(
		input_files=paths,
		file_metadata=get_metadata
	).load_data()
	parser = MarkdownNodeParser()
	nodes = parser.get_nodes_from_documents(docs)
	display(Markdown(paths[0]))
	return nodes, docs

In [10]:
df = category_files_df
cat = "tutorial"
qtd_docs = 1
nodes, docs = check_parser(category_files_df, cat, qtd_docs)

# Create Custom Project Templates<a name="sagemaker-projects-templates-custom"></a>

If the SageMaker\-provided templates do not meet your needs \(for example, you want to have more complex orchestration in the CodePipeline with multiple stages or custom approval steps\), create your own templates\.

We recommend starting by using SageMaker\-provided templates to understand how to organize your code and resources and build on top of it\. To do this, after you enable administrator access to the SageMaker templates, log in to the [https://console\.aws\.amazon\.com/servicecatalog/](https://console.aws.amazon.com/servicecatalog/), choose **Portfolios**, then choose **Imported**\. For information about Service Catalog, see [Overview of Service Catalog](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/what-is_concepts.html) in the *Service Catalog User Guide*\.

Create your own project templates to customize your MLOps project\. SageMaker project templates are Service Catalog–provisioned products to provision the resources for your MLOps project\. 

To create a custom project template, complete the following steps\.

1. Create a portfolio\. For information, see [Step 3: Create an Service Catalog Portfolio](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/getstarted-portfolio.html)\.

1. Create a product\. A product is a CloudFormation template\. You can create multiple versions of the product\. For information, see [Step 4: Create an Service Catalog Product](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/getstarted-product.html)\.

   For the product to work with SageMaker projects, add the following parameters to your product template\.

   ```
   SageMakerProjectName:
   Type: String
   Description: Name of the project
   
   SageMakerProjectId:
   Type: String
   Description: Service generated Id of the project.
   ```
**Important**  
We recommend that you wrap the CodeCommit repository into the SageMaker code repository for the project's repositories to be visible in VPC mode\. The sample template and required addition are shown in the following code samples\.  
Original \(sample\) template:  

   ```
   ModelBuildCodeCommitRepository:
       Type: AWS::CodeCommit::Repository
       Properties:
         # Max allowed length: 100 chars
         RepositoryName: !Sub sagemaker-${SageMakerProjectName}-${SageMakerProjectId}-modelbuild # max: 10+33+15+10=68
         RepositoryDescription: !Sub SageMaker Model building workflow infrastructure as code for the Project ${SageMakerProjectName}
         Code:
           S3:
             Bucket: SEEDCODE_BUCKETNAME
             Key: toolchain/model-building-workflow-v1.0.zip
           BranchName: main
   ```
Additional content to add in VPC mode:  

   ```
   SageMakerRepository:
       Type: AWS::SageMaker::CodeRepository
       Properties:
           GitConfig:
               RepositoryUrl: !GetAtt ModelBuildCodeCommitRepository.CloneUrlHttp
               Branch: main
   ```

1. Add a launch constraint\. A launch constraint designates an IAM role that Service Catalog assumes when a user launches a product\. For information, see [Step 6: Add a Launch Constraint to Assign an IAM Role](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/getstarted-launchconstraint.html)\.

1. Provision the product on [https://console\.aws\.amazon\.com/servicecatalog/](https://console.aws.amazon.com/servicecatalog/) to test the template\. If you are satisfied with your template, continue to the next step to make the template available in Studio\.

1. Grant access to the Service Catalog portfolio that you created in step 1 to your Studio execution role\. Use either the Studio domain execution role or a user role that has Studio access\. For information about adding a role to the portfolio, see [Step 7: Grant End Users Access to the Portfolio](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/getstarted-deploy.html)\.

1. To make your project template available in your **Organization templates** list in Studio, create a tag with the following key and value to the Service Catalog product you created in step 2\.
   + **key**: `sagemaker:studio-visibility`
   + **value**: `true`

After you complete these steps, Studio users in your organization can create a project with the template you created by following the steps in [Create an MLOps Project using Amazon SageMaker Studio](sagemaker-projects-create.md) and choosing **Organization templates** when you choose a template\.

In [12]:
docs[0].metadata

{'file_name': 'sagemaker-projects-templates-custom.md', 'category': 'tutorial'}

In [11]:
print(f"Parsed into {len(nodes)} nodes (chunks).")
for i, node in enumerate(nodes[:10]):
	print(f"--- Node {i+1} ---")
	print(f"Metadata: {node.metadata}")
	print(f"Content: {node.get_content()[:5000]}...")
	print("-" * 20)

Parsed into 1 nodes (chunks).
--- Node 1 ---
Metadata: {'file_name': 'sagemaker-projects-templates-custom.md', 'category': 'tutorial', 'header_path': '/'}
Content: # Create Custom Project Templates<a name="sagemaker-projects-templates-custom"></a>

If the SageMaker\-provided templates do not meet your needs \(for example, you want to have more complex orchestration in the CodePipeline with multiple stages or custom approval steps\), create your own templates\.

We recommend starting by using SageMaker\-provided templates to understand how to organize your code and resources and build on top of it\. To do this, after you enable administrator access to the SageMaker templates, log in to the [https://console\.aws\.amazon\.com/servicecatalog/](https://console.aws.amazon.com/servicecatalog/), choose **Portfolios**, then choose **Imported**\. For information about Service Catalog, see [Overview of Service Catalog](https://docs.aws.amazon.com/servicecatalog/latest/adminguide/what-is_concepts.

# Build Index

In [3]:
import os
import boto3
import logging
import sys
import pandas as pd
import config as cfg
import weave
from pathlib import Path
from typing import NoReturn

# llama-index imports
from llama_index.core import (
	VectorStoreIndex,
	SimpleDirectoryReader,
	StorageContext,
	Settings,
)
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini

# Other libraries
import qdrant_client
from pydantic import BaseModel, Field

In [18]:
%%pybash
docker run -d -p 6333:6333 -p 6334:6334 qdrant/qdrant

9ec1410b7a131fa602a7e743856245c1a9a1162f857d8e02027cd59f5ec9c1d5


In [89]:
import logging
from typing import Any, Dict, List, Optional, Union

category_files_df = load_obj(cfg.path.data.interim / "category_files_df_v1.pickle", as_df=True)

weave.init("aws_doc_ragqa_demo")

logger = logging.getLogger("llamaindex")
logging.basicConfig(level=logging.INFO)

def get_category(df: pd.DataFrame, path: Union[Path, str]) -> str:
	"""
	Retrieve the category for a given file path from the dataframe.

	Args:
		df (pd.DataFrame): DataFrame containing 'path' and 'category' columns.
		path (Union[Path, str]): The file path to look up.

	Returns:
		str: The category associated with the file path.

	Raises:
		IndexError: If the path is not found in the DataFrame.
	"""
	category = df[df.path == Path(path)]["category"].values[0]
	logger.debug(f"Category for {path}: {category}")
	return category

def get_metadata(file_path: Union[Path, str]) -> Dict[str, str]:
	"""
	Generate metadata dictionary for a given file path.

	Args:
		file_path (Union[Path, str]): The file path.

	Returns:
		Dict[str, str]: Metadata including file name and category.
	"""
	file_path = Path(file_path)
	meta = {"file_name": file_path.name, "category": get_category(category_files_df, file_path)}
	logger.debug(f"Metadata for {file_path}: {meta}")
	return meta

def get_nodes(df: pd.DataFrame) -> List[Any]:
	"""
	Parse markdown documents into nodes using the provided DataFrame.

	Args:
		df (pd.DataFrame): DataFrame with a 'path' column.

	Returns:
		List[Any]: List of parsed nodes.
	"""
	logger.info("Loading documents and parsing nodes...")
	paths = df.path.tolist()
	docs = SimpleDirectoryReader(
		input_files=paths,
		file_metadata=get_metadata
	).load_data()
	parser = MarkdownNodeParser()
	nodes = parser.get_nodes_from_documents(docs)
	logger.info(f"Parsed {len(nodes)} nodes from {len(paths)} documents.")
	return nodes

def set_llm_aws() -> None:
	"""
	Initialize Bedrock LLM and Embedding model for AWS and set them in Settings.
	"""
	llm_model_id = cfg.app.aws.llm_model_id
	embed_model_id = cfg.app.aws.embedding_model_id
	region_name = cfg.app.aws.region
	aws_session_token = os.environ["AWS_BEARER_TOKEN_BEDROCK"]
	
	logger.info(f"Setting AWS LLM: {llm_model_id}, Embedding: {embed_model_id}")
	llm = Bedrock(
		model=llm_model_id,
		region_name=region_name,
		aws_session_token=aws_session_token
	)
	embed_model = BedrockEmbedding(
		model_name=embed_model_id,
		region_name=region_name,
		aws_session_token=aws_session_token
	)
	Settings.llm = llm
	Settings.embed_model = embed_model

def set_llm_gemini() -> None:
	"""
	Initialize Gemini LLM and Embedding model and set them in Settings.
	"""
	llm_model_id = cfg.app.gemini.llm_model_id
	embed_model_id = cfg.app.gemini.embedding_model_id
	
	logger.info(f"Setting Gemini LLM: {llm_model_id}, Embedding: {embed_model_id}")
	llm = Gemini(model=llm_model_id)
	embed_model = GeminiEmbedding(model_name=embed_model_id)
	Settings.llm = llm
	Settings.embed_model = embed_model

def get_model_provider():
	llm_model = Settings.llm.model
	embed_model = Settings.embed_model.model_name
	
	if (llm_model == cfg.app.aws.llm_model_id) and (embed_model == cfg.app.aws.embedding_model_id):
		return "aws"
	elif (llm_model == cfg.app.gemini.llm_model_id) and (embed_model == cfg.app.gemini.embedding_model_id):
		return "gemini"
	else:
		return None
	
def set_model_provider(model_provider: str) -> None:
	cur_model_provider = get_model_provider()
	if model_provider != cur_model_provider:
		logger.info(f"The current model is: {cur_model_provider}")
		if model_provider == "aws":
			logger.info("Setting AWS Model...")
			set_llm_aws()
		elif model_provider == "gemini":
			logger.info("Setting Gemini Model...")
			set_llm_gemini()
		else:
			logger.warning(f"Unknown model provider: {model_provider}. Defaulting to AWS.")
			set_llm_aws()
	else:
		logger.info("The Model provider is already set.")

def qdrant_vector_store(model_provider: str = "aws") -> QdrantVectorStore:
	"""
	Create a QdrantVectorStore instance using configuration.
	
	Args:
		model_provider (str, optional): Model provider, "aws" or "gemini". Defaults to "aws".
	Returns:
		QdrantVectorStore: The vector store instance.
	"""
	collection_name = getattr(cfg.app.qdrant.collection, model_provider)
	url = cfg.app.qdrant.url
	
	logger.info(f"Connecting to Qdrant at {url}, collection: {collection_name}")
	client = qdrant_client.QdrantClient(url=url)
	vector_store = QdrantVectorStore(
		client=client, 
		collection_name=collection_name
	)
	return vector_store

def check_collection_exists(model_provider: str = "aws") -> bool:
	"""
	Check if the Qdrant collection exists.
	Args:
		model_provider (str, optional): Model provider, "aws" or "gemini". Defaults to "aws".

	Returns:
		bool: True if the collection exists, False otherwise.
	"""
	collection_name = getattr(cfg.app.qdrant.collection, model_provider)
	url = cfg.app.qdrant.url
	client = qdrant_client.QdrantClient(url=url)
	collection_exists = client.collection_exists(collection_name=collection_name)
	if collection_exists:
		logger.info(f"Collection '{collection_name}' exists!")
	else:
		logger.info(f"Collection '{collection_name}' does not exist!")
	return collection_exists

def build_index(
	vector_store: QdrantVectorStore,
	nodes: List[Any],
	model_provider: str = "aws",
	force_reindex: bool = False
) -> VectorStoreIndex:
	"""
	Build or load a VectorStoreIndex from Qdrant, setting up the LLM and embedding model.

	Args:
		vector_store (QdrantVectorStore): The vector store instance.
		nodes (List[Any]): List of nodes to index.
		model_provider (str, optional): Model provider, "aws" or "gemini". Defaults to "aws".
		force_reindex (bool, optional): If True, force reindexing. Defaults to False.

	Returns:
		VectorStoreIndex: The index instance.
	"""
	logger.info(f"Building index with model provider: {model_provider}, force_reindex={force_reindex}")
	collections_exists = check_collection_exists(model_provider)

	# Setup LLM llamaindex settings
	set_model_provider()

	if not force_reindex and collections_exists:
		logger.info("Index already exists. Loading from Qdrant.")
		index = VectorStoreIndex.from_vector_store(vector_store)
	else:
		logger.info("Building a new index in Qdrant.")
		storage_context = StorageContext.from_defaults(
			vector_store=vector_store,
		)
		index = VectorStoreIndex(
			nodes, 
			storage_context=storage_context,
			show_progress=True,
		)
	return index

def get_index(
	df: pd.DataFrame, 
	model_provider: str = "aws", 
	force_reindex: bool = False
) -> VectorStoreIndex:
	"""
	Get or build a VectorStoreIndex for the given DataFrame.

	Args:
		df (pd.DataFrame): DataFrame with file paths and categories.
		model_provider (str, optional): Model provider, "aws" or "gemini". Defaults to "aws".
		force_reindex (bool, optional): If True, force reindexing. Defaults to False.

	Returns:
		VectorStoreIndex: The index instance.
	"""
	logger.info("Preparing to get or build index...")
	nodes = get_nodes(df)
	vector_store = qdrant_vector_store(model_provider)
	index = build_index(vector_store, nodes, model_provider, force_reindex)
	logger.info("Index ready.")
	return index

In [57]:
# AWS Indexing
# Collection = sagemaker_docs_v1
df = category_files_df.copy()
aws_index = get_index(df)
# aws_index = get_index(df, force_reindex=True)

INFO:llamaindex:Preparing to get or build index...
INFO:llamaindex:Loading documents and parsing nodes...
Generating embeddings:   0%|          | 9/1826 [2:03:24<415:15:30, 822.75s/it]
INFO:llamaindex:Parsed 1826 nodes from 336 documents.
INFO:llamaindex:Connecting to Qdrant at http://localhost:6333, collection: sagemaker_docs_v1
INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/sagemaker_docs_v1/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/sagemaker_docs_v1 "HTTP/1.1 200 OK"
INFO:llamaindex:Building index with model provider: aws, force_reindex=False
INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/sagemaker_docs_v1/exists "HTTP/1.1 200 OK"
INFO:llamaindex:Collection 'sagemaker_docs_v1' exists!
INFO:llamaindex:Setting AWS LLM: us.anthropic.claude-3-haiku-20240307-v1:0, Embedding: 

In [43]:
# Gemini Indexing
# Collection = sagemaker_docs_v1.1
df = category_files_df.copy()
gemini_index = get_index(df, model_provider="gemini")

INFO:llamaindex:Preparing to get or build index...
INFO:llamaindex:Loading documents and parsing nodes...
Generating embeddings:   0%|          | 9/1826 [03:58<13:22:01, 26.48s/it]
INFO:llamaindex:Parsed 1826 nodes from 336 documents.
INFO:llamaindex:Connecting to Qdrant at http://localhost:6333, collection: sagemaker_docs_v1.1
INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/sagemaker_docs_v1.1/exists "HTTP/1.1 200 OK"
INFO:llamaindex:Building index with model provider: gemini, force_reindex=False
INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/sagemaker_docs_v1.1/exists "HTTP/1.1 200 OK"
INFO:llamaindex:Collection 'sagemaker_docs_v1.1' does not exist!
INFO:llamaindex:Setting Gemini LLM: models/gemini-1.5-flash, Embedding: models/embedding-001
INFO:llamaindex:Building a new index in Qdrant.
Generating embeddings: 100%|██

#### Query Index

In [115]:
def query_index(index: VectorStoreIndex, model_provider: str, query: str) -> None:
    """
    Query the index and print the response and sources.

    Args:
        index (VectorStoreIndex): The index to query.
        query (str): The query string.

    Returns:
        None
    """
    logger.info(f"Querying index: {query}")
    set_model_provider(model_provider)
    query_engine = index.as_query_engine(similarity_top_k=5)
    response = query_engine.query(query)
    print(f"\nQ: {query}")
    print(f"A: {response.response}")
    print("Sources:")
    for node in response.source_nodes:
        file_name = node.metadata.get('file_name', 'N/A')
        category = node.metadata.get('category', 'N/A')
        text = node.text
        score = node.score
        print(f"  - File: {file_name} \nCategory: {category} \nScore: {score:.4f}")
        print(f"  - Text: {text}")
        print('-' * 20)
        print()
    print('-' * 100)
    print('-' * 100)
    print('-' * 100)
    print()
    print()

In [93]:
from config import load_config
cfg = load_config()

In [111]:
q = "What is the Maximum length of the 'Value' property for a SageMakerPipelineParameter in an Events Rule?"
a = query_index(index=aws_index, model_provider="aws", query=q)
a

INFO:llamaindex:Querying index: What is the Maximum length of the 'Value' property for a SageMakerPipelineParameter in an Events Rule?


INFO:llamaindex:The Model provider is already set.
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1/points/search "HTTP/1.1 200 OK"



Q: What is the Maximum length of the 'Value' property for a SageMakerPipelineParameter in an Events Rule?
A: The maximum length of the 'Value' property for a SageMakerPipelineParameter in an Events Rule is 1024 characters.
Sources:
  - File: aws-properties-events-rule-sagemakerpipelineparameter.md 
Category: properties 
Score: 0.6366
  - Text: ### JSON<a name="aws-properties-events-rule-sagemakerpipelineparameter-syntax.json"></a>

```
{
  "[Name](#cfn-events-rule-sagemakerpipelineparameter-name)" : String,
  "[Value](#cfn-events-rule-sagemakerpipelineparameter-value)" : String
}
```
--------------------

  - File: aws-properties-events-rule-sagemakerpipelineparameter.md 
Category: properties 
Score: 0.6119
  - Text: ### YAML<a name="aws-properties-events-rule-sagemakerpipelineparameter-syntax.yaml"></a>

```
  [Name](#cfn-events-rule-sagemakerpipelineparameter-name): String
  [Value](#cfn-events-rule-sagemakerpipelineparameter-value): String
```
--------------------

  - File: aws-pr

In [112]:
questions = cfg.templates.questions
questions

['What is SageMaker?',
 'What are all AWS regions where SageMaker is available?',
 'How to check if an endpoint is KMS encrypted?',
 'What are SageMaker Geospatial capabilities?']

In [116]:
# AWS
for i, q in enumerate(questions):
    query_index(index=aws_index, model_provider="aws", query=q)

INFO:llamaindex:Querying index: What is SageMaker?
INFO:llamaindex:The Model provider is already set.
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: What are all AWS regions where SageMaker is available?
INFO:llamaindex:The Model provider is already set.



Q: What is SageMaker?
A: Amazon SageMaker is a fully managed service that enables developers and data scientists to build, train, and deploy machine learning models. It provides a comprehensive platform for all stages of the machine learning lifecycle, from data preparation to model deployment. SageMaker aims to simplify the process of creating and managing machine learning solutions, making it accessible to a wide range of users regardless of their level of expertise in machine learning.
Sources:
  - File: examples-sagemaker.md 
Category: tutorial 
Score: 0.6962
  - Text: # Working with Amazon SageMaker<a name="examples-sagemaker"></a>

 Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning \(ML\) models\. See the following resources for complete code examples with instructions\.

 [Link to Github](https://github.com/awsdocs/aws-doc-sdk-examples/tree/master/javav2/example_code/sagemak

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: How to check if an endpoint is KMS encrypted?
INFO:llamaindex:The Model provider is already set.



Q: What are all AWS regions where SageMaker is available?
A: I apologize, but I don't have information about the specific AWS regions where Amazon SageMaker is available based on the given context. The provided information focuses on general descriptions of SageMaker and its integration with AWS Marketplace, but does not include details about regional availability. For the most up-to-date and accurate information on SageMaker's regional availability, you would need to check the official AWS documentation or contact AWS support directly.
Sources:
  - File: examples-sagemaker.md 
Category: tutorial 
Score: 0.5683
  - Text: # Working with Amazon SageMaker<a name="examples-sagemaker"></a>

 Amazon SageMaker is a fully managed service that provides every developer and data scientist with the ability to build, train, and deploy machine learning \(ML\) models\. See the following resources for complete code examples with instructions\.

 [Link to Github](https://github.com/awsdocs/aws-doc-sdk

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: What are SageMaker Geospatial capabilities?
INFO:llamaindex:The Model provider is already set.



Q: How to check if an endpoint is KMS encrypted?
A: To check if an Amazon SageMaker endpoint is encrypted using a KMS key, you can use the SAGEMAKER_ENDPOINT_CONFIGURATION_KMS_KEY_CONFIGURED rule. This rule periodically checks whether a KMS key is configured for SageMaker endpoint configurations.

The rule will evaluate as NON_COMPLIANT if the 'KmsKeyId' is not specified for the SageMaker endpoint configuration. This means that if an endpoint configuration is found without a KMS key specified, it will be flagged as non-compliant.

You can also optionally specify a list of allowed KMS key ARNs using the 'kmsKeyArns' parameter. This allows you to ensure that only specific KMS keys are being used for encryption.

This check is performed periodically across most AWS regions, helping you maintain consistent encryption practices for your SageMaker endpoints.
Sources:
  - File: sagemaker-endpoint-configuration-kms-key-configured.md 
Category: security 
Score: 0.6241
  - Text: # sagemaker\-en

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1/points/search "HTTP/1.1 200 OK"



Q: What are SageMaker Geospatial capabilities?
A: SageMaker Geospatial capabilities are a set of features within Amazon SageMaker that allow users to perform geospatial operations. These capabilities operate as a managed service, meaning that SageMaker executes operations on behalf of users using AWS-managed hardware. 

To utilize these capabilities, users need to grant specific permissions through an IAM role, often referred to as an execution role. This role defines what operations SageMaker is allowed to perform on the user's behalf. 

It's important to note that SageMaker can only carry out operations that have been explicitly permitted by the user through this execution role. Users have the option to create and use a locally available execution role to manage these permissions effectively.
Sources:
  - File: sagemaker-geospatial-roles.md 
Category: security 
Score: 0.6410
  - Text: # SageMaker geospatial capabilities roles<a name="sagemaker-geospatial-roles"></a>

As a managed se

In [117]:
# GEMINI
for i, q in enumerate(questions):
    query_index(index=gemini_index, model_provider="gemini", query=q)

INFO:llamaindex:Querying index: What is SageMaker?
INFO:llamaindex:The current model is: aws
INFO:llamaindex:Setting Gemini Model...
INFO:llamaindex:Setting Gemini LLM: models/gemini-1.5-flash, Embedding: models/embedding-001
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1.1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: What are all AWS regions where SageMaker is available?
INFO:llamaindex:The Model provider is already set.



Q: What is SageMaker?
A: Amazon SageMaker is a fully managed service that enables developers and data scientists to build, train, and deploy machine learning (ML) models.

Sources:
  - File: sagemaker-projects-whatis.md 
Category: concepts 
Score: 0.8262
  - Text: ## Do I Need to Create a Project to Use SageMaker Pipelines?<a name="sagemaker-projects-need"></a>

No\. SageMaker pipelines are standalone entities just like training jobs, processing jobs, and other SageMaker jobs\. You can create, update, and run pipelines directly within a notebook by using the SageMaker Python SDK without using a SageMaker project\.

Projects provide an additional layer to help you organize your code and adopt operational best practices that you need for a production\-quality system\.
--------------------

  - File: sagemaker-controls.md 
Category: concepts 
Score: 0.8023
  - Text: # Amazon SageMaker controls<a name="sagemaker-controls"></a>

These controls are related to SageMaker resources\.
---------

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1.1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: How to check if an endpoint is KMS encrypted?
INFO:llamaindex:The Model provider is already set.



Q: What are all AWS regions where SageMaker is available?
A: This question cannot be answered from the given source.

Sources:
  - File: sagemaker-marketplace.md 
Category: concepts 
Score: 0.8007
  - Text: ## Topics<a name="sagemaker-marketplace-topics"></a>
+ [SageMaker Algorithms](#sagemaker-mkt-algorithm)
+ [SageMaker Model Packages](#sagemaker-mkt-model-package)
+ [Sell Amazon SageMaker Algorithms and Model Packages](sagemaker-marketplace-sell.md)
+ [Find and Subscribe to Algorithms and Model Packages on AWS Marketplace](sagemaker-mkt-find-subscribe.md)
+ [Use Algorithm and Model Package Resources](sagemaker-mkt-buy.md)
--------------------

  - File: sagemaker-marketplace.md 
Category: concepts 
Score: 0.7882
  - Text: ## SageMaker Model Packages<a name="sagemaker-mkt-model-package"></a>

Buyers use a model package to build a deployable model in SageMaker\. They can use the deployable model for real\-time inference by using SageMaker hosting services\. Or, they can get inference

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1.1/points/search "HTTP/1.1 200 OK"
INFO:llamaindex:Querying index: What are SageMaker Geospatial capabilities?
INFO:llamaindex:The Model provider is already set.



Q: How to check if an endpoint is KMS encrypted?
A: This question cannot be answered from the given source.  The provided text describes how to configure KMS keys for various AWS SageMaker resources using CloudFormation templates, but it does not explain how to check if an endpoint is already encrypted with a KMS key.

Sources:
  - File: sagemaker-endpoint-configuration-kms-key-configured.md 
Category: security 
Score: 0.7552
  - Text: ## AWS CloudFormation template<a name="w2aac12c33c15b9d525c15"></a>

To create AWS Config managed rules with AWS CloudFormation templates, see [Creating AWS Config Managed Rules With AWS CloudFormation Templates](aws-config-managed-rules-cloudformation-templates.md)\.
--------------------

  - File: sagemaker-notebook-instance-kms-key-configured.md 
Category: security 
Score: 0.7448
  - Text: ## AWS CloudFormation template<a name="w2aac12c33c15b9d529c15"></a>

To create AWS Config managed rules with AWS CloudFormation templates, see [Creating AWS Config

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/sagemaker_docs_v1.1/points/search "HTTP/1.1 200 OK"



Q: What are SageMaker Geospatial capabilities?
A: SageMaker geospatial capabilities are a managed service that operates on AWS hardware managed by SageMaker.  They only perform operations explicitly permitted by the user, which are granted through an IAM execution role.

Sources:
  - File: sagemaker-geospatial-roles.md 
Category: security 
Score: 0.7848
  - Text: # SageMaker geospatial capabilities roles<a name="sagemaker-geospatial-roles"></a>

As a managed service, Amazon SageMaker geospatial capabilities perform operations on your behalf on the AWS hardware that is managed by SageMaker\. It can perform only operations that the user permits\.

A user can grant these permissions with an IAM role \(referred to as an execution role\)\. 

To create and use a locally available execution role, you can use the following procedures\.
--------------------

  - File: sagemaker-projects-whatis.md 
Category: concepts 
Score: 0.7590
  - Text: ## Do I Need to Create a Project to Use SageMaker Pip